# Project Airbnb (Step 1)
---

## Problem statement

My project aims to help people set the best possible price for their Airbnb listing. Using different types of regression models, the model can predict what the most standard price will be for a new listing based on various factors, such as location, rooms, number of accommodations, etc.

Using natural language processing (NLP) analysis, I collected common verbs and text from a large sample of existing Airbnb listings in New York. This data is useful when predicting the price of listings.

With the help of time series models like ARIMA and fbprophet, analyze if there are any kind of trends or seasons in relation to the price. At the same try to predict the average price over time.

## Airbnb

Airbnb is an online marketplace that connects people who want to rent out their homes with people who are looking for accommodations in specific locales.

## Data gathering 

This data was taken from http://insideairbnb.com/get-the-data.html. The data behind the Inside Airbnb site is sourced from publicly available information from the Airbnb site. For the purpose of this project, we will use the data from New York City listings from February 6 and reservations from 2021 to 2022.

## Process

This project will be divided into 3 parts: 
 - Data cleaning 
 - EDA analysis 
 - Models and predictions.

## Agenda

1. Imports
2. Data reading
3. Null checking  
4. Data preparation 
5. Conclusion 

### 1. Imports:
---

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
pd.set_option('display.max_columns', None) # to see all the columns on my dataframe
listing= pd.read_csv('../data/raw_data/listing.csv',low_memory=False)
review =pd.read_csv('../data/raw_data/reviews.csv')
calendar2022 =pd.read_csv('../data/raw_data/calendar.csv')
calendar2021 =pd.read_csv('../data/raw_data/calendar2021feb.csv')

### 2. Data reading:
---

In [6]:
listing.head(3)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,20220106025017,2022-01-06,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Centrally located in the heart of Manhattan ju...,https://a0.muscache.com/pictures/f0813a11-40b2...,2845,https://www.airbnb.com/users/show/2845,Jennifer,2008-09-09,"New York, New York, United States",A New Yorker since 2000! My passion is creatin...,within a day,51%,17%,f,https://a0.muscache.com/im/pictures/user/50fc5...,https://a0.muscache.com/im/pictures/user/50fc5...,Midtown,8.0,8.0,"['email', 'phone', 'reviews', 'offline_governm...",t,t,"New York, United States",Midtown,Manhattan,40.75356,-73.98559,Entire rental unit,Entire home/apt,1,NaN,1 bath,NaN,1.0,"[""Hot water"", ""Iron"", ""Keypad"", ""Refrigerator""...",$150.00,30,1125,30.0,30.0,1125.0,1125.0,30.0,1125.0,NaN,t,30,60,90,334,2022-01-06,48,0,0,2009-11-21,2019-11-04,4.70,4.72,4.62,4.76,4.79,4.86,4.41,NaN,f,3,3,0,0,0.33
1,3831,https://www.airbnb.com/rooms/3831,20220106025017,2022-01-06,"Whole flr w/private bdrm, bath & kitchen(pls r...","Enjoy 500 s.f. top floor in 1899 brownstone, w...",Just the right mix of urban center and local n...,https://a0.muscache.com/pictures/e49999c2-9fd5...,4869,https://www.airbnb.com/users/show/4869,LisaRoxanne,2008-12-07,"New York, New York, United States",Laid-back Native New Yorker (formerly bi-coast...,a few days or more,0%,64%,f,https://a0.muscache.com/im/users/4869/profile_...,https://a0.muscache.com/im/users/4869/profile_...,Clinton Hill,1.0,1.0,"['email', 'phone', 'reviews', 'offline_governm...",t,t,"Brooklyn, New York, United States",Bedford-Stuyvesant,Brooklyn,40.68494,-73.95765,Entire guest suite,Entire home/apt,3,NaN,1 bath,1.0,3.0,"[""Hot water"", ""Iron"", ""Children\u2019s books a...",$73.00,1,730,1.0,1.0,730.0,730.0,1.0,730.0,NaN,t,3,15,26,214,2022-01-06,409,26,0,2014-09-30,2021-10-22,4.45,4.58,4.49,4.78,4.80,4.71,4.64,NaN,f,1,1,0,0,4.62
2,5121,https://www.airbnb.com/rooms/5121,20220106025017,2022-01-06,BlissArtsSpace!,<b>The space</b><br />HELLO EVERYONE AND THANK...,NaN,https://a0.muscache.com/pictures/2090980c-b68e...,7356,https://www.airbnb.com/users/show/7356,Garon,2009-02-03,"New York, New York, United States","I am an artist(painter, filmmaker) and curato...",within an hour,100%,100%,f,https://a0.muscache.com/im/pictures/user/72a61...,https://a0.muscache.com/im/pictures/user/72a61...,Bedford-Stuyvesant,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'off...",t,t,NaN,Bedford-Stuyvesant,Brooklyn,40.68535,-73.95512,Private room in rental unit,Private room,2,NaN,NaN,1.0,1.0,"[""Heating"", ""Air conditioning"", ""Long term sta...",$60.00,30,730,30.0,30.0,730.0,730.0,30.0,730.0,NaN,t,30,60

In [7]:
review.head(1)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2595,17857,2009-11-21,50679,Jean,Notre séjour de trois nuits.\r<br/>Nous avons ...


In [8]:
calendar2021.head(5)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,21456,2021-02-05,t,$85.00,$85.00,30.0,365.0
1,59709,2021-02-05,f,$160.00,$160.00,7.0,31.0
2,59709,2021-02-06,f,$160.00,$160.00,7.0,31.0
3,59709,2021-02-07,f,$160.00,$160.00,7.0,31.0
4,59709,2021-02-08,f,$160.00,$160.00,7.0,31.0


In [7]:
calendar2022.head(1)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,2595,2022-01-06,t,$150.00,$150.00,30.0,1125.0


In [8]:
print('listings shape: ',listing.shape)
print('review shape: ', review.shape)
print('calendar 2021 shape: ', calendar2021.shape)
print('calendar 2022 shape: ', calendar2022.shape)

listings shape:  (38185, 74)
review shape:  (908803, 6)
calendar 2021 shape:  (13464021, 7)
calendar 2022 shape:  (13930969, 7)


In [9]:
listing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38185 entries, 0 to 38184
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            38185 non-null  int64  
 1   listing_url                                   38185 non-null  object 
 2   scrape_id                                     38185 non-null  int64  
 3   last_scraped                                  38185 non-null  object 
 4   name                                          38171 non-null  object 
 5   description                                   37077 non-null  object 
 6   neighborhood_overview                         22560 non-null  object 
 7   picture_url                                   38185 non-null  object 
 8   host_id                                       38185 non-null  int64  
 9   host_url                                      38185 non-null 

In [10]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908803 entries, 0 to 908802
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     908803 non-null  int64 
 1   id             908803 non-null  int64 
 2   date           908803 non-null  object
 3   reviewer_id    908803 non-null  int64 
 4   reviewer_name  908797 non-null  object
 5   comments       908638 non-null  object
dtypes: int64(3), object(3)
memory usage: 41.6+ MB


In [11]:
calendar2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13464021 entries, 0 to 13464020
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       object 
 3   price           object 
 4   adjusted_price  object 
 5   minimum_nights  float64
 6   maximum_nights  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 719.1+ MB


In [12]:
calendar2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13930969 entries, 0 to 13930968
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       object 
 3   price           object 
 4   adjusted_price  object 
 5   minimum_nights  float64
 6   maximum_nights  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 744.0+ MB


### 3. Null checking
---

In [13]:
# Checking for null values
null_colum = listing.columns[listing.isnull().any()]
listing[null_colum].isnull().sum()

#https://dzone.com/articles/pandas-find-rows-where-columnfield-is-null

name                              14
description                     1108
neighborhood_overview          15625
host_name                         47
host_since                        47
host_location                    149
host_about                     16313
host_response_time             14290
host_response_rate             14290
host_acceptance_rate           13310
host_is_superhost                 47
host_thumbnail_url                47
host_picture_url                  47
host_neighbourhood              7424
host_listings_count               47
host_total_listings_count         47
host_has_profile_pic              47
host_identity_verified            47
neighbourhood                  15624
bathrooms                      38185
bathrooms_text                   103
bedrooms                        3889
beds                            1287
minimum_minimum_nights            18
maximum_minimum_nights            18
minimum_maximum_nights            18
maximum_maximum_nights            18
m

In [14]:
null_colum = review.columns[review.isnull().any()]
review[null_colum].isnull().sum()

reviewer_name      6
comments         165
dtype: int64

In [15]:
null_colum = calendar2021.columns[calendar2021.isnull().any()]
calendar2021[null_colum].isnull().sum()

price               170
adjusted_price      170
minimum_nights    15293
maximum_nights    15293
dtype: int64

In [16]:
null_colum = calendar2022.columns[calendar2022.isnull().any()]
calendar2022[null_colum].isnull().sum()

price              5
adjusted_price     5
minimum_nights    21
maximum_nights    21
dtype: int64

### 4. Data preparation
---

*  Dropping unnecessary columns:
---

In [17]:
listing.drop(['scrape_id','host_thumbnail_url','host_picture_url','host_listings_count','picture_url','host_total_listings_count','host_since',
              'host_verifications','host_identity_verified','bathrooms','host_has_profile_pic','neighbourhood',
              'availability_30','availability_60','availability_90','availability_365',
             'calendar_last_scraped','number_of_reviews_ltm','number_of_reviews_l30d','license','instant_bookable',
             'calculated_host_listings_count','calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms',
             'calculated_host_listings_count_shared_rooms','host_response_time','host_response_rate','host_acceptance_rate','minimum_minimum_nights',
             'maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm',
             'calendar_updated','first_review','last_review','last_scraped','host_url','host_neighbourhood','minimum_nights',
             'maximum_nights','has_availability'],axis=1,inplace=True)

calendar2021.drop('adjusted_price',axis=1,inplace=True)
calendar2021.dropna(subset=['price'],inplace=True)
calendar2021.dropna(subset=['minimum_nights'],inplace=True)

calendar2022.drop('adjusted_price',axis=1,inplace=True)
calendar2022.dropna(subset=['price'],inplace=True)
calendar2022.dropna(subset=['minimum_nights'],inplace=True)

#dropping values price = 0
listing.drop(listing[listing['price']==0].index,inplace=True)


* Filling Null values:
---

In [18]:
listing['name'].fillna(str(listing['id']), inplace=True)
listing['description'].fillna('None', inplace=True)
listing['neighborhood_overview'].fillna('None', inplace=True)
listing['host_name'].fillna(str(listing['id']), inplace=True)
listing['host_location'].fillna('New York, New York, United States', inplace=True)
listing['host_about'].fillna('None', inplace=True)
listing['host_is_superhost'].fillna('f', inplace=True)
listing['bathrooms_text'].fillna('1 bath', inplace=True)
listing['bedrooms'].fillna('1.0', inplace=True)
listing['beds'].fillna(listing['beds'].mode()[0], inplace=True)


review['reviewer_name'].fillna('None', inplace=True)
review['comments'].fillna('None', inplace=True)

* Data transformation:
---

In [19]:
listing['price'] = listing['price'].replace('[\$\,]', '', regex=True).astype(float)

calendar2021['price'] = calendar2021['price'].replace('[\$\,]', '', regex=True).astype(float)

calendar2022['price'] = calendar2022['price'].replace('[\$\,]', '', regex=True).astype(float)


#https://stackoverflow.com/questions/44469313/price-column-object-to-int-in-pandas

In [20]:
# Change date column to be datetime dtype

review['date'] = pd.to_datetime(review['date'])
calendar2021['date'] = pd.to_datetime(calendar2021['date'])
calendar2022['date'] = pd.to_datetime(calendar2022['date'])

In [21]:
listing['host_is_superhost'].replace(to_replace=['t', 'f'], value=[1, 0],inplace=True)
calendar2021['available'].replace(to_replace=['t', 'f'], value=[1, 0],inplace=True)
calendar2022['available'].replace(to_replace=['t', 'f'], value=[1, 0],inplace=True)

In [22]:
# creating a new column bathroom 
listing['bathrooms_text']=listing['bathrooms_text'].str.lower()
listing['bathroom'] = listing['bathrooms_text'].replace('[\ bath\ baths\ shared\ private\ Half-]', '', regex=True)
listing['bathroom'] = listing['bathroom'].replace('','0.5',regex=True).astype(float)

In [23]:
# Dropping price values = 0
listing.drop(listing[listing['price']==0].index,inplace=True)

### 5. Creating dataframe
---

In [24]:
calendar2021= calendar2021[calendar2021['date']<='2022-01-31']
calendar2022= calendar2022[calendar2022['date']>='2022-02-01']

In [25]:
frames = [calendar2021,calendar2022]

reservations_df = pd.concat(frames)

In [26]:
reservations_df  = reservations_df[reservations_df['date']<='2023-01'] # Taking the dates from Feb 2021 to dec 2022

In [27]:
# creatiang a new column weekday name
reservations_df = reservations_df.groupby('date').agg({'available':'sum','price':'mean'}).reset_index()
reservations_df['weekday'] = reservations_df['date'].dt.day_name()
reservations_df.drop([0],inplace=True) # Dropping first row
reservations_df.set_index('date',inplace=True)

In [30]:
reservations_df

,available,price,weekday
date,,,
2021-02-05,3847,149.310480,Friday
2021-02-06,9204,145.043317,Saturday
2021-02-07,11242,142.344009,Sunday
2021-02-08,12248,142.182521,Monday
2021-02-09,12496,142.167703,Tuesday
...,...,...,...
2022-12-28,13181,177.325255,Wednesday
2022-12-29,13164,178.026148,Thursday
2022-12-30,13133,182.766028,Friday


### 6. Save dataframes
---

In [28]:
print('listings shape: ',listing.shape)
print('review shape: ', review.shape)
print('reservations_df shape: ', reservations_df.shape)

listings shape:  (38149, 32)
review shape:  (908803, 6)
reservations_df shape:  (696, 3)


In [29]:
listing.to_json('../data/listing_clean.json')
review.to_json('../data/review_clean.json')
reservations_df.to_json('../data/reservations_clean.json')


### 5. Conclusion 
---

In total 3 dataframes have been saved:
- Listing: Which contains all the information of the listings and the hosts, from the start of Airbnb in 2008 until the month of February 2022. This dataframe started with 38185 rows and 74 columns, and after cleaning and collecting the best features, it was reduced to 38149 rows with 32 columns.
- Review: This dataframe has the information of the reviews of the listings.
- Reservations_df: Here you will find the union of the two dataframes calendar_2021 and calendar_2022, with a size greater than 13000000 rows and 7 columns. this new dataframe contains the average price and the sum of the daily availability.